# Review Generation

In [28]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer,  AutoConfig, AutoModelForNextSentencePrediction
from langchain import PromptTemplate
from langchain.llms import OpenAI

## Model

Product name = ""\
0:Very bad\
1:Bad\
2:Not so good\
3:Acceptable\
4:Good\
5:Very good\

In [2]:
# transforming score into adjectives
criteria = [{"0":"Very bad", "1":"Bad", "2":"Not so good", "3":"Acceptable", "4":"Good", "5":"Very good"}]

In [77]:
#example
product_name = "iphone"

#fake values
score = [0, 1, 2, 3, 4]
criteria_score = [{"Design and Build Quality":score[0], "Display":score[1], "Performance":score[2], "Camera Quality":score[3], "Battery Life":score[4]}]

In [29]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

True

## API

In [19]:
from openai import OpenAI
client = OpenAI()

product_name = "iphone"
criteria = "camera"
feelings = "very bad"

# Create a prompt including criteria and feelings
prompt = f"Generate a product review about the {criteria} of {product_name}  that you find {feelings}."

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "As a costumer"},
    {"role": "user", "content": prompt}
  ]
)

In [20]:
response.choices[0].message.content

"I recently purchased the latest model of the iPhone, and I must say I am extremely disappointed with the camera quality. Despite the high price tag and the brand's reputation for top-notch technology, the camera on this device has been a major letdown.\n\nThe images captured are often blurry and lack sharpness, even in well-lit environments. The colors appear muted and washed out, making pictures look dull and unimpressive. Additionally, the low-light performance is subpar, resulting in grainy and noisy photos that are far from the crisp and clear images I was expecting.\n\nThe overall user experience with the camera app is also frustrating, with slow autofocus and shutter lag causing missed moments and blurred shots. The lack of manual controls and limited customization options further hinder the camera's performance, making it difficult to achieve the desired results.\n\nIn conclusion, I would not recommend the iPhone camera to anyone looking for high-quality photography capabilitie

## Langchain

In [26]:
from langchain import PromptTemplate
from langchain.llms import OpenAI

In [30]:
llm = OpenAI(temperature=0.9) #model_name = "text-davinci-003"

We can use prompt template to:
+  Intruct the Language Model
+ set a few shot examples to help the Language model generate a better response
+ ask questions to the Language model

In [41]:
template = "{adjective} {product_name}: My {adjective} {product_name} has one of the {adjective} {criteria} of her generation.\
            I have used this {product_name} for YEARS and I still {feelings}"

In [ ]:
prompt = PromptTemplate(
    input_variables = [ "product_name", "criteria", "adjective", "feelings"],
    template = template
)

llm(prompt.format(product_name="iphone", criteria="camera", adjective="bad", feelings = "not happy"))

In [17]:
# Criteria, Adjective and Feeling lists

criteria_list, adjective_list, feeling_list = [f"criteria_{n}" for n in range(len(criteria_score[0]))],\
                                              [f"adjective_{n}" for n in range(len(criteria_score[0]))],\
                                              [f"feeling_{n}" for n in range(len(criteria_score[0]))]

In [16]:
# Dictionary score
Criteria_Score = [{}]

for  value, key in enumerate(criteria_list):
    Criteria_Score[0][key] = score[value]

Criteria_Score

[{'criteria_0': 0,
  'criteria_1': 1,
  'criteria_2': 2,
  'criteria_3': 3,
  'criteria_4': 4}]

In [ ]:
prefix = []
for key, value in Criteria_Score[0].items():
    pref = f"{key}:{criteria[0][str(Criteria_Score[0][key])]}"
    prefix.append(pref)

In [65]:
# template for PromptTemplate()

def template(prefix, criteria, Criteria_Score):
    """ 
        This function build a template with a set of instructions to be passed to the model.
    """
    temp = f" You are a costumer. I want you to generate a product review on {product_name} considering  the following criteria: {prefix}. Be precise and concise."
    
    return temp

In [66]:
template = template(criteria, criteria_score)
template

" You are a costumer. I want you to generate a product review on iphone considering  the following criteria: ['Design and Build Quality:Very bad', 'Display:Bad', 'Performance:Not so good', 'Camera Quality:Acceptable', 'Battery Life:Good']. Be precise and concise."

In [72]:
#input variables

def input_variable(product_name, criteria_list,  adjective_list,  feeling_list=None,  **kwargs):
    """ 
        This function builds the input_variables necessary for the PromptTemplate() 
        Here, we have made feeling_list optional. input_variable() can be called without it!
    """    
    if feeling_list is not None:
        input_var = [f"{product_name}"] + criteria_list + adjective_list + feeling_list
    else: 
        input_var = [f"{product_name}"] + criteria_list + adjective_list
        
    return input_var

In [75]:
input_variables = input_variable(product_name, criteria_list,  adjective_list)
print(input_variables)

['iphone', 'criteria_0', 'criteria_1', 'criteria_2', 'criteria_3', 'criteria_4', 'adjective_0', 'adjective_1', 'adjective_2', 'adjective_3', 'adjective_4']


In [108]:
# building the prompt

prompt = PromptTemplate(
    input_variables = input_variables,
    template = template
)

In [109]:
input_criteria = []
score = []
for key, value in criteria_score[0].items():
    input_criteria.append(key)
    score.append(value)

In [110]:
def Prompt(criteria_0, criteria_1, criteria_2, criteria_3, criteria_4, criteria_5=None, criteria_6=None, criteria_7=None, criteria_8=None, **kwargs):
    response = llm(prompt.format(kwargs))
    return response

In [107]:
Prompt(product_name="iphone", criteria_0="Design and Build Quality", criteria_1="Display", criteria_2="Performance", \
                  criteria_3="Camera Quality", criteria_4="Battery Life", adjective_0="Very Bad", adjective_1="Bad",\
                  adjective_2="Acceptable", adjective_3="Good", adjective_4="Very Good")

TypeError: PromptTemplate.format() takes 1 positional argument but 2 were given

In [ ]:
product_name="iphone", criteria_0="Design and Build Quality", criteria_1="Display", criteria_2="Performance", \
                  criteria_3="Camera Quality", criteria_4="Battery Life", adjective_0="Very Bad", adjective_1="Bad",\
                  adjective_2="Acceptable", adjective_3="Good", adjective_4="Very Good"

In [182]:
llm(prompt.format(product_name="iphone", criteria_0="Design and Build Quality", criteria_1="Display", criteria_2="Performance", \
                  criteria_3="Camera Quality", criteria_4="Battery Life", adjective_0="Very Bad", adjective_1="Bad",\
                  adjective_2="Acceptable", adjective_3="Good", adjective_4="Very Good"))

"\n\nI have to say, I am extremely disappointed with the design and build quality of the iPhone. It feels cheap and flimsy in my hand, and I'm worried about it breaking easily. The display is also a major letdown. The resolution is not as sharp as I would expect from a high-end phone, and the colors seem washed out.\n\nAs for performance, it's not terrible, but it's definitely not great. I've noticed some lag and slow loading times, which is frustrating when I'm trying to use my phone quickly. And while the camera quality is acceptable, it's nothing to write home about. The pictures are decent, but they don't have the same crispness and detail as other smartphones on the market.\n\nOn a positive note, the battery life of the iPhone is surprisingly good. I can go a full day without having to charge it, even with heavy usage. But unfortunately, that's about the only positive thing I have to say about this phone.\n\nOverall, I would not recommend the iPhone to anyone looking for a top-of-

## To be Discussed

+ Given that a product review is very subjective, we could ask the user if he/she wants to provide a recommendation.
+ It is better to avoid negativity.
+ The user should meet a few but import prerequisites. For example, one shouldn't review his own product or any product from similar company. In other words the user should be related in one way or the other with the seller of manufacturer.